In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import h5py
from scipy.stats import rankdata
import pandas as pd
import glob
%matplotlib inline

In [ ]:
f.close()
p.close()

In [ ]:
sp = 'Mesculenta'
group = 'plants'

In [ ]:
sp = 'Mtruncatula'

In [ ]:
sp = 'Tcacao'

In [ ]:
sp = 'papio_anubis'
group = 'animals'

In [ ]:
sp = 'equus_caballus'

In [ ]:
sp = 'petromyzon_marinus'

In [ ]:

path = glob.glob('../../../{}/single*/{}/test_data.h5'.format(group, sp))[0]
f = h5py.File(path)

In [ ]:
path = glob.glob('../../../../predictions/{}/ensemble_*/{}/predictions.h5'.format(group, sp))[0]
p = h5py.File(path)

In [ ]:
f['data/X'].shape[0]

In [ ]:

# select examples of interest
# just look at first 10k chunks to avoid mem errors and speed things up
startenough = 0_000
enough = 10_000 + startenough

# mask to skip anything with padding
chunk_size = f['data/X'].shape[1]
print(chunk_size)
unpadded = np.sum(np.sum(f['data/X'][startenough:enough].astype(float), axis=2), axis=1) == chunk_size


In [ ]:
# mask to skip anything that's just big and totally ig
anynotig = np.argmax(f['data/y'][startenough:enough], axis=2) + np.argmax(p['predictions'][startenough:enough], axis=2) > 0
notig10 = np.sum(anynotig, axis=1) / chunk_size > 0.1

# percent accuracy
right = np.argmax(f['data/y'][startenough:enough], axis=2) == np.argmax(p['predictions'][startenough:enough], axis=2)
acc = np.sum(right, axis=1) / chunk_size



In [ ]:
# track ori indexes
h5i = np.arange(startenough, enough, 1)

In [ ]:
mask = unpadded & notig10
h5i = h5i[mask]
rankacc = rankdata(acc[mask])
rankscore = rankdata(f['scores/one_centered'][startenough:enough][mask])

In [ ]:
upper = h5i.shape[0] * 0.9
lower = h5i.shape[0] * 0.1

#Low accuracy, high ref score
candidates = h5i[(rankacc < lower) & (rankscore > upper)]
#Low accuracy, low ref score
candidates = h5i[(rankacc < lower) & (rankscore < lower)]
#High accuracy, high ref score
candidates = h5i[(rankacc > upper) & (rankscore > upper)]

#High accuracy, low ref score
candidates = h5i[(rankacc > upper) & (rankscore < lower)]
print(len(candidates))
print(candidates)

In [ ]:
h5i.shape

In [ ]:
j = -1


In [ ]:
j += 1
i = candidates[j]
print(i)


print(f['data/seqids'][i], f['data/start_ends'][i])
fig, (ax1, ax2, axaug, ax3) = plt.subplots(4, 1, sharex=True, figsize=(14,7),
                                         gridspec_kw={"height_ratios": [2, 0.5, 0.5, 0.5]})
#plt.figure(figsize=(3,10))

ax1.plot(np.log(f['evaluation/coverage'][i] + 1), c='black')
ax1.plot(np.log(f['evaluation/spliced_coverage'][i] + 1), c='grey', linestyle=':')
ax1.set_xlim((0, chunk_size))
ax1.set_ylabel("ln(coverage + 1)")
ax1.legend(['cov', 'sc'])

yticks = ['', 'i-genic', 'utr', 'cds', 'intron']
ax2.imshow(np.array(f['data/y'][i].T).astype(float), aspect="auto", interpolation='none')
ax2.set_yticklabels(yticks)
ax2.set_ylabel('reference')

axaug.imshow(np.array(f['alternative/augustus/y'][i].T).astype(float), aspect="auto", interpolation='none')
utrmarker = np.array([1] * 20000)
axaug.plot(utrmarker, linestyle=(0, (10, 5)), c='black')
axaug.set_yticklabels(yticks)
axaug.set_ylabel('augustus')

ax3.imshow(np.array(p['predictions'][i].T).astype(float), aspect="auto", interpolation='none')
ax3.set_yticklabels(yticks)
ax3.set_ylabel('helixer')


In [ ]:
utrmarker[:3] + 1

In [ ]:
print(get_seq(i=i, f=f, start=9900, end=10100))
print(get_seq(i=i, f=f, start=11200, end=12000))

In [ ]:
def plot_selected(i, f, p, start, end, figsize=(7, 4.2), as_int=True, legend=False, c_augline='gainsboro'):
    print(i)
    print(f['data/seqids'][i], f['data/start_ends'][i])
    fig, (ax1, ax2, axaug, ax3) = plt.subplots(4, 1, sharex=True, figsize=figsize,
                                             gridspec_kw={"height_ratios": [1.2, 0.5, 0.5, 0.5]})
    #plt.figure(figsize=(3,10))

    ax1.plot(np.log(f['evaluation/coverage'][i][start:end] + 1), c='black')
    ax1.plot(np.log(f['evaluation/spliced_coverage'][i][start:end] + 1), c='dimgrey', linestyle=':')
    ax1.set_xlim((0, end - start))
    ax1.set_ylabel("ln(coverage + 1)")
    if legend:
        ax1.legend(['cov', 'sc'])

    yticks = ['IG', 'UTR', 'CDS', 'Ntrn']
    ax2.imshow(1 - np.array(f['data/y'][i][start:end].T).astype(float), aspect="auto", 
               cmap='gray', interpolation='none')
    ax2.set_ylim([-0.5, 3.5])
    ax2.set_yticks([0,1,2, 3])
    ax2.set_yticklabels(yticks)
    ax2.set_ylabel('Ref.')

    axaug.imshow(1 - np.array(f['alternative/augustus/y'][i][start:end].T).astype(float), aspect="auto", 
                 cmap='gray', interpolation='none')
    utrmarker = np.array([1] * 20000)
    axaug.plot(utrmarker, linestyle=(0, (10, 5)), c=c_augline)
    axaug.set_ylim([-0.5, 3.5])
    axaug.set_yticks([0,1,2, 3])
    axaug.set_yticklabels(yticks)
    axaug.set_ylabel('Aug.')

    ax3.imshow(1 - np.array(p['predictions'][i][start:end].T).astype(float), aspect="auto", 
               cmap='gray', interpolation='none')
    ax3.set_ylim([-0.5, 3.5])
    ax3.set_yticks([0,1,2, 3])
    ax3.set_yticklabels(yticks)
    ax3.set_ylabel('Helixer')
    se = f['data/start_ends'][i]

    at = [x * (end - start) for x in [0, .2, .4, .60, .8, 1]]
    ax3.set_xticks(at)
    offset = f['data/start_ends'][i][0] + start
    if se[1] - se[0] > 0:
        sign = '+'
        xticklabs = [(offset + x)/ 10**3 for x in at]
    else:
        sign = '-'
        xticklabs = [(offset - x)/ 10**3 for x in at]
    if as_int:
        xticklabs = [int(x) for x in xticklabs]
    else:
        xticklabs = [round(x, 2) for x in xticklabs]
    ax3.set_xticklabels(xticklabs)
    ax3.set_xlabel('position on ({}) strand of {} in kbp'.format(sign, f['data/seqids'][i].decode()))
    

In [ ]:
# petromyzon_marinus
i = 3166
start, end = 8000, 9000
plot_selected(i, f, p, start, end, figsize=(15, 4.2), as_int=False, legend=True)
plt.savefig(fname='zooms/petromyzon_marinus3166.eps')


In [ ]:
# equus_caballus
i = 691
start, end = 91200, 92200
plot_selected(i, f, p, start, end, figsize=(15, 4.2), as_int=False, legend=True)
plt.savefig(fname='zooms/equus_caballus691.eps')
i = 981
start, end = 12500, 13500
plot_selected(i, f, p, start, end, figsize=(15, 4.2), as_int=False, legend=True)
plt.savefig(fname='zooms/equus_caballus981.eps')

In [ ]:
# papio_anubis 
i = 479
start, end = 78800, 79800
plot_selected(i, f, p, start, end, figsize=(15, 4.2), as_int=False, legend=True)

plt.savefig(fname='zooms/papio_anubis479.eps')

In [ ]:
# Mtruncatula 
i = 146
start, end = 17800, 18800
plot_selected(i, f, p, start, end, figsize=(15, 4.2), as_int=False, legend=True)
plt.savefig(fname='zooms/Mtruncatula146.eps')
i = 1
start, end = 5600, 6600
plot_selected(i, f, p, start, end, figsize=(15, 4.2), as_int=False, legend=True)
plt.savefig(fname='zooms/Mtruncatula1.eps')

In [ ]:
# Tcacao 
i = 1602
start, end = 7000, 8000
plot_selected(i, f, p, start, end, figsize=(15, 4.2), as_int=False, legend=True)
plt.savefig(fname='zooms/Tcacao1602.eps')

In [ ]:
# Mesculenta
i = 3378
start, end = 18300, 19300
plot_selected(i, f, p, start, end, figsize=(15, 4.2), as_int=False, legend=True)
plt.savefig(fname='zooms/Mesculenta3378.eps')


In [ ]:
## Mesculenta
# high score, low acc
i = 6436
start, end = 0, 10000
plot_selected(i, f, p, start, end)
plt.savefig(fname='highScore_lowAcc_001.eps')

i = 5392
start, end = 7000, 17000
plot_selected(i, f, p, start, end)
plt.savefig(fname='highScore_lowAcc_002.eps')

i = 1162
start, end = 6000, 16000
plot_selected(i, f, p, start, end)
plt.savefig(fname='highScore_lowAcc_003.eps')

In [ ]:
# low score, low acc
# looks good
i = 1024
start, end = 10000, 20000
plot_selected(i, f, p, start, end)
plt.savefig(fname='lowScore_lowAcc_001.eps')

i = 392
start, end = 0, 10000
plot_selected(i, f, p, start, end)
plt.savefig(fname='lowScore_lowAcc_002.eps')


# errors to go around
i = 432
start, end = 10000, 20000
plot_selected(i, f, p, start, end)
plt.savefig(fname='lowScore_lowAcc_003.eps')


In [ ]:
# high acc high score

i = 18
start, end = 5000, 15000
plot_selected(i, f, p, start, end)
plt.savefig(fname='highScore_highAcc_001.eps')


i = 7113
start, end = 2500,12500
plot_selected(i, f, p, start, end)
plt.savefig(fname='highScore_highAcc_002.eps')


i = 3233
start, end = 0, 10000
plot_selected(i, f, p, start, end)
plt.savefig(fname='highScore_highAcc_003.eps')


In [ ]:
# high acc, low score

# ghost gene
i = 5569
start, end = 5000, 15000
plot_selected(i, f, p, start, end, legend=True)
plt.savefig(fname='lowScore_highAcc_001.eps')

# missed intron?
i = 5967
plot_selected(i, f, p, start, end, legend=True)
plt.savefig(fname='lowScore_highAcc_002.eps')

# spliced read mapping error
i = 9936
start, end = 3000, 13000
plot_selected(i, f, p, start, end, legend=True)
plt.savefig(fname='lowScore_highAcc_003.eps')


In [ ]:
def get_seq(i, f, start, end):
    bps = 'catg'
    argmaxes = np.argmax(f['data/X'][i][start:end], axis=1)
    return ''.join([bps[c] for c in argmaxes])